# Pull request scraper

## GitHub credentials
A private access token is necessary to make use of less restrictive API limits.

In [3]:
from github import RateLimitExceededException, Github

# Providing access token
access_token = ""
g = Github(login_or_token=access_token)

# Confirm your login is successful
user = g.get_user()
print(f"Authenticated as: {user.login}")

Authenticated as: AbelvdTil


## Files

In [4]:
import os
 
STEP4_TFCOMMITS = os.path.join("data", "step4-tf-commits.json") 
STEP5_TF_REPOS_WITH_PR = os.path.join("data", "step5-tf-repos-with-pr.json")
STEP6_TF_REPOS_COMMITS = os.path.join("data", "step6-tf-repos-commits.json")
STEP7_TF_REPOS_WITH_TF_PR = os.path.join("data", "step7-tf-repos-with-tf-pr.json")
STEP8_TF_KEYWORD_PR = os.path.join("data", "step8-tf-keyword-pr.json")

## File seperator and combinator

Files over 100MB are not stored on GitHub, therefore we need to seperate large files into smaller ones.
Any step5 and step7 files can be split into multiple smaller ones and be combined together.

The repositories are split on pull request level. The first pull request is stored in part 1, the 2nd in part 2 etc. This will make sure that it is fairly equally distributed.

Each pull request is accomodated with the url of the repository. Therefore it is possible to reconstruct the original file again.

In [27]:
import json
import math

FILE_TO_SEPERATE = STEP7_TF_REPOS_WITH_TF_PR

nr_parts = 4

### File seperator

In [28]:
parts_data = []

for i in range(nr_parts):
    parts_data.append([])

file = open(FILE_TO_SEPERATE)
seperator_data = json.load(file)

count = 0
for repository in seperator_data:
    for pull_request in repository["pull_requests"]:
        part = (count % nr_parts)
        count += 1
        parts_data[part].append({"repo_url": repository["url"], "pull_request": pull_request})

for i in range(nr_parts):
    with open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(i+1) + ".json", "w") as outfile:
        json.dump(parts_data[i], outfile)

39062


### File combinator

Also reverts changes in data structure 

From:

{
    [
        {
        "url": string,
        "pull_request": { PR }
    ]
}

To:

{
    [
        {
        "url": string,
        "pull_requests": [ { PR } ]
    ]
}

In [9]:
url_dict = {}
combinator_data = []
index_count = 0
for part in range(nr_parts):
    part_file = open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(part+1) + ".json", "r")
    part_data = json.load(part_file)

    for nugget in part_data:
        # find repo using url
        index = url_dict.get(nugget["repo_url"], None)

        if (index == None):
            repo = {"url": nugget["repo_url"], "pull_requests": []}
            combinator_data.append(repo)
            
            url_dict[nugget["repo_url"]] = index_count
            index_count += 1
        else:
            repo = combinator_data[index]
            
        repo["pull_requests"].append(nugget["pull_request"])

with open(FILE_TO_SEPERATE, "w") as outfile:
    json.dump(combinator_data, outfile) 

## STEP 5: Pull request scraping script

For each repository, get all pull request data. This includes PR Title, description, (review) comments and commit hashes. Exclude any repositories that do not have pull requests.

In [3]:
# SETTINGS
check_limit_every_x_calls = 5
api_limit_buffer = 10
api_calls_per_debug = 500

# INITIALIZATION

import json
import datetime
import time

# INITIALIZATION
terraform_output = open(STEP4_TFCOMMITS)
step4_output = json.load(terraform_output)

# Retrieve data from previous run
try:
    previous_run = open(STEP5_TF_REPOS_WITH_PR)
    repoData_dict = json.load(previous_run)
except FileNotFoundError as e:
    repoData_dict = []

iteration = 0
calls_till_next_debug = 0
calls_till_limit_checkup = 0

# Check for api limits, also periodically calls print debug.
def CheckForApiLimit():
    global calls_till_limit_checkup
    global calls_till_next_debug
    global api_calls_per_debug
    global api_limit_buffer

    # check for limit
    if (calls_till_limit_checkup == 0):
        core_limit = g.get_rate_limit().core

        # sleep when exceeded api core limit
        if (core_limit.remaining <= api_limit_buffer):
            time_to_sleep = core_limit.raw_data['reset'] - time.time() + 1
            print("Rate limit exceeded, sleeping for", time_to_sleep, "seconds.", "Actual remaining calls", core_limit.remaining)
            time.sleep(time_to_sleep)

        calls_till_limit_checkup = check_limit_every_x_calls
    
    calls_till_limit_checkup -= 1

    # check for debug
    if (calls_till_next_debug == 0):
        PrintDebug()
        calls_till_next_debug = api_calls_per_debug

    calls_till_next_debug -= 1

# Prints debug message
def PrintDebug():
    global iteration
    global repo_url

    print(datetime.datetime.now().strftime("%H:%M:%S"), ":", 
              "current iteration:", iteration, 
              "url:", repo_url)

# Pull request scraping script
for rp in step4_output["repositories"]:
    try:
        iteration += 1
            
        repo_url = rp["name"]

        # skip already scraped repositories
        if any(d["url"] == repo_url for d in repoData_dict):
            continue

        # Get the repo object from the url
        split_list = repo_url.split("/")
        actual_url = (split_list[3]+ '/' + split_list[4]).split('.git')[0]
        repo = g.get_repo(actual_url)
        
        # Get required info for pull requests
        pull_requests_dict = []
        pull_requests = repo.get_pulls(state="closed")

        if len(pull_requests) != 0:
            for pr in repo.get_pulls(state="closed"):
    
                # retrieve all review comments, not required if there are none.
                comments = []
                if (pr.review_comments > 0):
                    for review in pr.get_reviews():
                        if (review.body.strip() != ""):
                            comments.append(review.body)
                    CheckForApiLimit()
    
                if (pr.comments > 0):
                    for comment in pr.get_issue_comments():
                        if (comment.body.strip() != ""):
                            comments.append(comment.body)
                    CheckForApiLimit()
    
                # retrieve all connected commits.
                commits = []
                for commit in pr.get_commits():
                    commits.append(commit.sha)
                CheckForApiLimit()
    
                pull_requests_dict.append({"url": pr.html_url, "title": pr.title, "body": pr.body, "comments": comments, "commits": commits})
            
            CheckForApiLimit()        
            repoData_dict.append({"url": repo_url, "pull_requests": pull_requests_dict});
            
            with open(STEP5_TF_REPOS_WITH_PR, "w") as outfile:
                json.dump(repoData_dict, outfile)
    except Exception as e:
        print("exception:", e)

exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message

KeyboardInterrupt: 

## STEP 6: Find tf commits for tf repos with pr's 

For remaining repositories from step 5, collect all commits that modify a terraform file.

In [14]:
from pydriller import Repository

import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
step5_dict = json.load(step5)

terraform_keywords = ['.tf', '.tf.json']

iteration = 0
    
# Pull request scraping script
repo_dic = []
for rp in step5_dict:
    try:
        if (iteration % 50 == 0):
            print("at iteration", iteration)
            with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
                json.dump(repo_dic, outfile)

        iteration += 1

        repo = Repository(rp["url"])

        # Get each commit
        commit_dic = []
        for commit in repo.traverse_commits():

            modified_terraform = False
            # find if it changes a terraform file
            for file in commit.modified_files:
                if any(key in file.filename for key in terraform_keywords):
                    modified_terraform = True
            
            if modified_terraform:
                commit_dic.append({"hash": commit.hash, 
                                   "url": rp["url"].split(".git")[0] + "/commit/" + commit.hash, 
                                   "date": str(commit.author_date), 
                                   "body": commit.msg})
  
        repo_dic.append({"url":rp["url"], "commits":commit_dic})
        
    except Exception as e:
        print("exception:", e)

with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
        json.dump(repo_dic, outfile)

at iteration 0
at iteration 50
at iteration 100
at iteration 150
at iteration 200
at iteration 250
at iteration 300
at iteration 350
at iteration 400
at iteration 450
at iteration 500
at iteration 550
at iteration 600


## STEP 7: filter out pull requests without tf commit

Removes any pull request that does not include a commit from the previous step, for the remaining pull requests, it combines the two datasets into one.

In [15]:
import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
repository_input = json.load(step5)

step6 = open(STEP6_TF_REPOS_COMMITS)
commit_input = json.load(step6)

iteration = 0

output_dict = []

# for each repository
for repository in repository_input:
    # find commits for repo from step 7
    commit_input_list = next(repo["commits"] for repo in commit_input if repo["url"] == repository["url"])

    pr_dict = []
    # for each pull request
    for pull_request in repository["pull_requests"]:
        commit_dict = []

        # for each commit
        for commit_hash in pr["commits"]:
            # Find the exact commit from step 7
            commit_data = next((commit for commit in commit_input_list if commit["hash"] == commit_hash), None)
            if (commit_data is not None):
                commit_dict.append(commit_data)

        pull_request["total_commits"] = len(pull_request["commits"])
        pull_request["commits"] = commit_dict
        
        if (len(commit_dict) > 0):
            pr_dict.append(pull_request)
    
    if (len(pr_dict) > 0):
        output_dict.append({"url": repository["url"], "pull_requests": pr_dict})

with open(STEP7_TF_REPOS_WITH_TF_PR, "w") as outfile:
    json.dump(output_dict, outfile) 

## STEP 8: list all tf pull request with a keyword

In [16]:
cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

pullrequest_dict_output = []
for repository in repo_input:
    for pr in repository["pull_requests"]:
        
        title   = True if (pr["title"]        is not None and any(key in pr["title"].lower()    for key in cost_keywords)) else False
        body    = True if (pr["body"]         is not None and any(key in pr["body"].lower()     for key in cost_keywords)) else False
        comment = True if (any(comment        is not None and     key in comment.lower()        for key in cost_keywords for comment in pr["comments"])) else False
        commit  = True if (any(commit["body"] is not None and     key in commit["body"].lower() for key in cost_keywords for commit  in pr["commits"]))  else False
            
        reason = (("title " if title else "") + 
                  ("body " if body else "") + 
                  ("comment " if comment else "") + 
                  ("commit " if commit else ""))
        
        if (title or body or comment or commit):
            pullrequest_dict_output.append({"reason": reason.strip(), "pull_request": pr})

with open(STEP8_TF_KEYWORD_PR, "w") as outfile:
    json.dump(pullrequest_dict_output, outfile) 

## STEP 9: results

In [26]:
import json

cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step4 = open(STEP4_TFCOMMITS)
step4_data = json.load(step4)

step5 = open(STEP5_TF_REPOS_WITH_PR)
step5_data = json.load(step5)

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

step8 = open(STEP8_TF_KEYWORD_PR)
pr_reason_input = json.load(step8)


# GENERAL REPOSITORY DATA

print("Total repositories:" , step4_data["no_of_repos"])
print("Existing repositories with pull requests:", len(step5_data))
print("Repositories with TF pull requests:", len(repo_input))

print("")

# GENERAL PULL REQUEST DATA

print("Total PR's with keywords found:", len(pr_reason_input))

print("")

print("PR with keyword in:\t", "Only in:")
print("Title:\t\t", len([pr for pr in pr_reason_input if "title" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "title" == pr["reason"]]))
print("Description:\t", len([pr for pr in pr_reason_input if "body" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "body" == pr["reason"]]))
print("Comment:\t", len([pr for pr in pr_reason_input if "comment" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "comment" == pr["reason"]]))
print("commit message:\t", len([pr for pr in pr_reason_input if "commit" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "commit" == pr["reason"]]))  

print("")

# GENERAL COMMIT DATA

print("Total amount of commits in terraform pr's:", sum([pr["total_commits"] for repo in repo_input for pr in repo["pull_requests"]]))
print("Total amount of terraform commits in terraform pr's:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"]]))
print("Terraform commits with a keyword in terraform pr's:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"] if any(key in commit["body"].lower() for key in cost_keywords)]))

print("")

count = 0
for repo in repo_input:
    for pr in repo["pull_requests"]:
        if (pr["total_commits"] >= 250):
            print(pr["url"])
            count += 1
print(count)
    


Total repositories: 1278
Existing repositories with pull requests: 610
Repositories with TF pull requests: 476


Total PR's with keywords found: 1135


PR with keyword in:	 Only in:
Title:		 283 	 8
Description:	 457 	 183
Comment:	 314 	 257
commit message:	 655 	 230


Total amount of commits in terraform pr's: 131521
Total amount of terraform commits in terraform pr's: 5883
Terraform commits with a keyword in terraform pr's: 862


https://github.com/ministryofjustice/cloud-platform-infrastructure/pull/345
https://github.com/Azure/sap-automation/pull/462
https://github.com/Azure/sap-automation/pull/421
https://github.com/Azure/sap-automation/pull/408
https://github.com/Azure/sap-automation/pull/372
https://github.com/Azure/sap-automation/pull/278
https://github.com/Azure/sap-automation/pull/407
https://github.com/Azure/sap-automation/pull/458
https://github.com/Azure/sap-automation/pull/276
https://github.com/Azure/sap-automation/pull/441
https://github.com/Azure/sap-automation/pull/